# Elliptic curve cryptography 
## Tutorial/playground (part 3)

In [ ]:
import * as Utils from './src/util';
import * as ed25519 from '@noble/ed25519';

Unlike part 1 and part 2 tutorials, this tutorial is not much about
the elliptic curve stuff. 

Instead, this is about proving systems.proving systems involves two characters: Prover and Verifier.

Prover has a secret and he must convince the Verifier about facts regarding the secret, WITHOUT revealing the secret.

There are two "versions" of proving systems: interactive and non-interactive.
* interactive
    * in the interactive version, Verifier is involved *during* the creation of full proof by Prover.
    * in cryptography papers, it is the interactive version hat is usually presented.
* non-interactive
    * in the non-interactive version, the full proof is created by Prover alone, and Verifier will only verify the full proof. 
    * in actual implementations of cryptography, it is the non-interactive version that is usually implemented. we'll cover both here.

### Exercise: implement Schnorr protocol

read more: https://en.wikipedia.org/wiki/Proof_of_knowledge#Schnorr_protocol

note: the wiki uses "multiplicative notation" for the group binary operation

(more common overall), but these tutorials (and Monero resources) use

"additive notation" (more commonly seen when dealing with elliptic curves)

Schnorr protocol is among the simplest proving system currently used!

here's the scenario:
* Prover has a secret scalar x. he sends the commitment `P = xG` to Verifier.
* by the Discrete Logarithm (DL) assumption (see part 1), Verifier will not be able to crack the value of x.
* however the Verifier still wants to be convinced that the Prover really knows x. How would the Prover do that?

Schnorr protocol allows Prover to do this!



### interactive

In [ ]:
class SchnorrProof {

   private G = ed25519.Point.BASE;
   private P: ed25519.Point;
   private c: Utils.Scalar;
   private Q: ed25519.Point;
   private s: Utils.Scalar;

   /**
    * Use SchnorrProof.init(scalar, point)
    * for async - await initialization.
    */
   constructor() {}
   public init = async (x: Utils.Scalar, P: ed25519.Point): Promise<void> => {
      // we won't store the secret x here
      this.P = P;
      // let this.Q = rG.

      // <-     code here       ->
      
      // now Prover would send P and Q to Verifier.
      // once Verifier receives P and Q, she gives
      // an interactive challenge c to Prover.
      this.c = await Utils.rnd_scalar();

      // once Prover received the challenge c, let this.s = r + c * x.
      
      // <-     code here       ->

      // Prover would send s to Verifier. This completes the full proof.
   }

    /**
     * once Verifier receives the full proof, she can now verify it.
     */
     public verify = async (): Promise<boolean> => {
        // return s * G == Q + c * P

        // <-     code here       ->

     }
     
}

### non-interactive

In [ ]:

class NISchnorrProof {

    private G = ed25519.Point.BASE;
    private P: ed25519.Point;
    private Q: Utils.Scalar;
    private s: Utils.Scalar;

    /**
    * Use SchnorrProof.init(scalar, point)
    * for async - await initialization.
    */
    constructor() {}
    public init = async (x: Utils.Scalar, P: ed25519.Point): Promise<void> => {
        // we won't store the secret x here
        this.P = P;
        // let r be a random scalar (don't put in this)
        // let this.Q = rG.

        // <-     code here       ->
        
        /* Unlike in interactive version, Prover must generate the challenge
           themmself. however, he should not be able to cheat by manipulating the
           challenge. hence, the challenge instead should be the hash of the
           partial proof data. this trick is called "Fiat-Shamir heuristic".
           c = hash_to_scalar("Schnorr Proof", this.P.get_hex_value(), this.Q.toHex())
           Not in this!
         */

        // let this.s = r + c * x.
        
        // <-     code here       ->

        // now Prover would send the full proof (P, Q, s) to Verifier.

    }

    /**
     * once Verifier receives the full proof, she can now verify it.
     */
     public verify = async (): Promise<boolean> => {
        //  s * G == Q + hash_to_scalar("Schnorr Proof", P, Q) * P

        // <-     code here       ->
     }

}

### Testing

In [ ]:

//test 1 (should work)
const prvkey = await Utils.rnd_scalar();
const pubkey = ed25519.Point.BASE.multiply(await prvkey.get_value());
    
const proof1 = new SchnorrProof();
await proof1.init(prvkey, pubkey);
// also try NISchnorrProof
if (await proof1.verify())
    console.log("Proof1 Verified!");
else
    console.log("Something's wrong (T_T)!");

// test 2 (should NOT work)
const prvkey2 = await Utils.rnd_scalar();
const s = new Utils.Scalar(BigInt("1"));
const s2 = await prvkey2.add(await s.get_value());
const pubkey2 = ed25519.Point.BASE.multiply(await s2.get_value());

const proof2 = new SchnorrProof();
await proof2.init(prvkey2, pubkey2);
// also try NISchnorrProof
if (await proof2.verify())
    console.log("Proof2 Verified!");
else
    console.log("Something's wrong (T_T)!")